In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("JoinExamples").getOrCreate()

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

schema2 = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("local", StringType(), True)
])

data1 = [(1, "Alice", 25, "New York"), (None, "Bob", 30, "London"), (3, "Charlie", 22, "Paris"), (4, "David", 28, "Tokyo")]
df1 = spark.createDataFrame(data1, schema=schema)

data2 = [(1, "Alice", 24, "New York","monday"), (5, "Eve", 24, "Sydney",None), (1, "Bob", 35, "London","Tuesday"), (None, "Frank", 35, "Berlin",None),(None, "kil", 35, "Berlin", "sunday")]  # Note: Some overlap in 'id' and 'name'
df2 = spark.createDataFrame(data2, schema=schema2)

df1.show()
df2.show()

25/02/02 07:52:15 WARN Utils: Your hostname, codespaces-56288b resolves to a loopback address: 127.0.0.1; using 10.0.2.66 instead (on interface eth0)
25/02/02 07:52:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/02 07:52:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/02 07:52:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+----+-------+---+--------+
|  id|   name|age|    city|
+----+-------+---+--------+
|   1|  Alice| 25|New York|
|NULL|    Bob| 30|  London|
|   3|Charlie| 22|   Paris|
|   4|  David| 28|   Tokyo|
+----+-------+---+--------+

+----+-----+---+--------+-------+
|  id| name|age|    city|  local|
+----+-----+---+--------+-------+
|   1|Alice| 24|New York| monday|
|   5|  Eve| 24|  Sydney|   NULL|
|   1|  Bob| 35|  London|Tuesday|
|NULL|Frank| 35|  Berlin|   NULL|
|NULL|  kil| 35|  Berlin| sunday|
+----+-----+---+--------+-------+



In [5]:
# left join
df3 = df1.join(df2, \
df1.id == df2.id, 'left')

df3.show()

+----+-------+---+--------+----+-----+----+--------+-------+
|  id|   name|age|    city|  id| name| age|    city|  local|
+----+-------+---+--------+----+-----+----+--------+-------+
|NULL|    Bob| 30|  London|NULL| NULL|NULL|    NULL|   NULL|
|   1|  Alice| 25|New York|   1|  Bob|  35|  London|Tuesday|
|   1|  Alice| 25|New York|   1|Alice|  24|New York| monday|
|   3|Charlie| 22|   Paris|NULL| NULL|NULL|    NULL|   NULL|
|   4|  David| 28|   Tokyo|NULL| NULL|NULL|    NULL|   NULL|
+----+-------+---+--------+----+-----+----+--------+-------+



In [4]:

# df3.show()

df7 = df1.join(df2, df1.id.eqNullSafe(df2.id), 'left')
df7.show()

#df8 = df1.union(df2)
#df8.show()

df9 = df1.unionByName(df2,allowMissingColumns=True)
df9.show()
# right join
 
df4 = df2.join(df1, df1.id == df2.id, 'right')
# df4.show()

# inner join 

df5 = df1.join(df2, df1.id == df2.id, 'inner' )
# df5.show()

df6 = df1.join(df2, df1.id == df2.id, 'outer')
# df6.show()

+----+-------+---+--------+----+-----+----+--------+-------+
|  id|   name|age|    city|  id| name| age|    city|  local|
+----+-------+---+--------+----+-----+----+--------+-------+
|   1|  Alice| 25|New York|   1|  Bob|  35|  London|Tuesday|
|   1|  Alice| 25|New York|   1|Alice|  24|New York| monday|
|NULL|    Bob| 30|  London|NULL|  kil|  35|  Berlin| sunday|
|NULL|    Bob| 30|  London|NULL|Frank|  35|  Berlin|   NULL|
|   4|  David| 28|   Tokyo|NULL| NULL|NULL|    NULL|   NULL|
|   3|Charlie| 22|   Paris|NULL| NULL|NULL|    NULL|   NULL|
+----+-------+---+--------+----+-----+----+--------+-------+

+----+-------+---+--------+-------+
|  id|   name|age|    city|  local|
+----+-------+---+--------+-------+
|   1|  Alice| 25|New York|   NULL|
|NULL|    Bob| 30|  London|   NULL|
|   3|Charlie| 22|   Paris|   NULL|
|   4|  David| 28|   Tokyo|   NULL|
|   1|  Alice| 24|New York| monday|
|   5|    Eve| 24|  Sydney|   NULL|
|   1|    Bob| 35|  London|Tuesday|
|NULL|  Frank| 35|  Berlin|  

25/02/02 07:52:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
